In [20]:
from rake_nltk import Rake   # ensure this is installed
import nltk
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import warnings
warnings.filterwarnings("ignore")

In [21]:
df = pd.read_csv('Codeacademy_Webscrapper_20210107154307.csv')   
df

,Unnamed: 0,Topic Title,Category,Tags,Leading Comment,Other Comments,Likes,Views
0,0,[How to] Format code in posts,Getting Started,[],Codecademy Discuss posts are parsed with Markd...,[],21,13.9k
1,1,Guide: A list of how-to’s for giving and recei...,Getting Started,[],Technical guides will teach you how to use the...,[],0,0
2,2,Guide: Learn about our community forums,Getting Started,[],Take a moment and read through each of these b...,[],0,0
3,3,[How-to] Create a topic that everyone will read,Getting Started,[],A great topic is one that provides potential h...,[],0,0
4,4,[Learn More] How the forum is organized,Getting Started,[],"With Codecademy Community forums, conversation...",[],0,0
...,...,...,...,...,...,...,...,...
5459,5459,Why does it ask me not to use compute_bill?,FAQ,['Python FAQ'],Question\nWhy does it ask me not to use comput...,['Question\nWhy does it ask me not to use comp...,15,8.7k
5460,5460,FAQ: Learn Python - Battleship - A Real Win,FAQ,['Python FAQ'],\nCommunity FAQs on Codecademy Python Exercise...,['\nCommunity FAQs on Codecademy Python Exerci...,0,1.2k
5461,5461,How should I indent the code to create my_tria...,FAQ,['Python FAQ'],Question\nHow should I indent the code to crea...,[],0,0
5462,5462,FAQ: Learn Python - Battleship - Not Again!,FAQ,['Python FAQ'],\nCommunity FAQs on Codecademy Python Exercise...,['\nCommunity FAQs on Codecademy Python Exerci...,12,1.4k


In [22]:
df = df[['Topic Title','Category','Tags','Leading Comment','Other Comments']]
df

,Topic Title,Category,Tags,Leading Comment,Other Comments
0,[How to] Format code in posts,Getting Started,[],Codecademy Discuss posts are parsed with Markd...,[]
1,Guide: A list of how-to’s for giving and recei...,Getting Started,[],Technical guides will teach you how to use the...,[]
2,Guide: Learn about our community forums,Getting Started,[],Take a moment and read through each of these b...,[]
3,[How-to] Create a topic that everyone will read,Getting Started,[],A great topic is one that provides potential h...,[]
4,[Learn More] How the forum is organized,Getting Started,[],"With Codecademy Community forums, conversation...",[]
...,...,...,...,...,...
5459,Why does it ask me not to use compute_bill?,FAQ,['Python FAQ'],Question\nWhy does it ask me not to use comput...,['Question\nWhy does it ask me not to use comp...
5460,FAQ: Learn Python - Battleship - A Real Win,FAQ,['Python FAQ'],\nCommunity FAQs on Codecademy Python Exercise...,['\nCommunity FAQs on Codecademy Python Exerci...
5461,How should I indent the code to create my_tria...,FAQ,['Python FAQ'],Question\nHow should I indent the code to crea...,[]
5462,FAQ: Learn Python - Battleship - Not Again!,FAQ,['Python FAQ'],\nCommunity FAQs on Codecademy Python Exercise...,['\nCommunity FAQs on Codecademy Python Exerci...


In [23]:
# data overview
print('Rows x Columns : ', df.shape[0], 'x', df.shape[1])
print('Features: ', df.columns.tolist())
print('\nUnique values:')
print(df.nunique())

Rows x Columns :  5464 x 5
Features:  ['Topic Title', 'Category', 'Tags', 'Leading Comment', 'Other Comments']

Unique values:
Topic Title        5189
Category              5
Tags                 65
Leading Comment    5463
Other Comments     4248
dtype: int64


In [24]:
# type of entries, how many missing values/null fields
df.info()
print('\nMissing values:  ', df.isnull().sum().values.sum())
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5464 entries, 0 to 5463
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Topic Title      5464 non-null   object
 1   Category         5464 non-null   object
 2   Tags             5464 non-null   object
 3   Leading Comment  5464 non-null   object
 4   Other Comments   5464 non-null   object
dtypes: object(5)
memory usage: 213.6+ KB

Missing values:   0


Topic Title        0
Category           0
Tags               0
Leading Comment    0
Other Comments     0
dtype: int64

## There is no missing  value

In [25]:
# summary statistics for all numerical columns
df.describe().T

,count,unique,top,freq
Topic Title,5464,5189,Orion Constellation Project,47
Category,5464,5,Projects,2130
Tags,5464,65,[],1670
Leading Comment,5464,5463,"When you ask a question, don’t forget to inclu...",2
Other Comments,5464,4248,[],1217


In [26]:
#to extract all categories in 1 list
df['Category'] = df['Category'].map(lambda x: x.split(','))
## create unique identity names by merging first & last name into one word, convert to lowercase 
for index, row in df.iterrows():
    row['Category'] = [x.lower().replace(' ','') for x in row['Category']]

df

,Topic Title,Category,Tags,Leading Comment,Other Comments
0,[How to] Format code in posts,[gettingstarted],[],Codecademy Discuss posts are parsed with Markd...,[]
1,Guide: A list of how-to’s for giving and recei...,[gettingstarted],[],Technical guides will teach you how to use the...,[]
2,Guide: Learn about our community forums,[gettingstarted],[],Take a moment and read through each of these b...,[]
3,[How-to] Create a topic that everyone will read,[gettingstarted],[],A great topic is one that provides potential h...,[]
4,[Learn More] How the forum is organized,[gettingstarted],[],"With Codecademy Community forums, conversation...",[]
...,...,...,...,...,...
5459,Why does it ask me not to use compute_bill?,[faq],['Python FAQ'],Question\nWhy does it ask me not to use comput...,['Question\nWhy does it ask me not to use comp...
5460,FAQ: Learn Python - Battleship - A Real Win,[faq],['Python FAQ'],\nCommunity FAQs on Codecademy Python Exercise...,['\nCommunity FAQs on Codecademy Python Exerci...
5461,How should I indent the code to create my_tria...,[faq],['Python FAQ'],Question\nHow should I indent the code to crea...,[]
5462,FAQ: Learn Python - Battleship - Not Again!,[faq],['Python FAQ'],\nCommunity FAQs on Codecademy Python Exercise...,['\nCommunity FAQs on Codecademy Python Exerci...


In [27]:
df['Tags'] = df['Tags'].apply(lambda x: x.lower()) #lowercasing tags
df

,Topic Title,Category,Tags,Leading Comment,Other Comments
0,[How to] Format code in posts,[gettingstarted],[],Codecademy Discuss posts are parsed with Markd...,[]
1,Guide: A list of how-to’s for giving and recei...,[gettingstarted],[],Technical guides will teach you how to use the...,[]
2,Guide: Learn about our community forums,[gettingstarted],[],Take a moment and read through each of these b...,[]
3,[How-to] Create a topic that everyone will read,[gettingstarted],[],A great topic is one that provides potential h...,[]
4,[Learn More] How the forum is organized,[gettingstarted],[],"With Codecademy Community forums, conversation...",[]
...,...,...,...,...,...
5459,Why does it ask me not to use compute_bill?,[faq],['python faq'],Question\nWhy does it ask me not to use comput...,['Question\nWhy does it ask me not to use comp...
5460,FAQ: Learn Python - Battleship - A Real Win,[faq],['python faq'],\nCommunity FAQs on Codecademy Python Exercise...,['\nCommunity FAQs on Codecademy Python Exerci...
5461,How should I indent the code to create my_tria...,[faq],['python faq'],Question\nHow should I indent the code to crea...,[]
5462,FAQ: Learn Python - Battleship - Not Again!,[faq],['python faq'],\nCommunity FAQs on Codecademy Python Exercise...,['\nCommunity FAQs on Codecademy Python Exerci...


In [28]:
df['Comment'] = df['Leading Comment'] + ' ' + df['Other Comments']
df['Comment']

0       Codecademy Discuss posts are parsed with Markd...
1       Technical guides will teach you how to use the...
2       Take a moment and read through each of these b...
3       A great topic is one that provides potential h...
4       With Codecademy Community forums, conversation...
                              ...                        
5459    Question\nWhy does it ask me not to use comput...
5460    \nCommunity FAQs on Codecademy Python Exercise...
5461    Question\nHow should I indent the code to crea...
5462    \nCommunity FAQs on Codecademy Python Exercise...
5463    Question\nWhy is my Triangle class not accepte...
Name: Comment, Length: 5464, dtype: object

In [35]:
# For regular expressions
import re
# For handling string
import string
# For performing mathematical operations
import math
from bs4 import BeautifulSoup
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\'\“\”\’\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
DIGITS_RE=re.compile('\w*\d\w*')


def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = text.replace(r'\n', ' ')
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = DIGITS_RE.sub('', text)
    return text

In [36]:
df['Comment'] = df['Comment'].apply(clean_text)

In [37]:
# to extract key words from Comments to a list
df['Key_words'] = ''   # initializing a new column
r = Rake()   # use Rake to discard stop words (based on english stopwords from NLTK)

for index, row in df.iterrows():
    r.extract_keywords_from_text(row['Comment'])   # to extract key words from Comments, default in lower case
    key_words_dict_scores = r.get_word_degrees()    # to get dictionary with key words and their scores
    row['Key_words'] = list(key_words_dict_scores.keys())   # to assign list of key words to new column

df.head()

,Topic Title,Category,Tags,Leading Comment,Other Comments,Comment,Key_words
0,[How to] Format code in posts,[gettingstarted],[],Codecademy Discuss posts are parsed with Markd...,[],codecademy discuss posts are parsed with markd...,"[small, amount, visible, either, wrap, span, a..."
1,Guide: A list of how-to’s for giving and recei...,[gettingstarted],[],Technical guides will teach you how to use the...,[],technical guides will teach you how to use the...,"[learn, use, format, code, forums, previous, a..."
2,Guide: Learn about our community forums,[gettingstarted],[],Take a moment and read through each of these b...,[],take a moment and read through each of these b...,"[community, guidelines, moment, take, learn, r..."
3,[How-to] Create a topic that everyone will read,[gettingstarted],[],A great topic is one that provides potential h...,[],a great topic is one that provides potential h...,"[title, icon, get, accurate, much, information..."
4,[Learn More] How the forum is organized,[gettingstarted],[],"With Codecademy Community forums, conversation...",[],with codecademy community forums conversation...,"[peers, subject, mattertopic, individual, entr..."


In [38]:
df['Comment'][2]

'take a moment and read through each of these bitesized posts to learn about our community forums learn more  welcome to the codecademy community forums  learn more  how the forum is organized  learn more  categories help keep the forum tidy learn more  about our community leaders  learn more  introducing yourself  the do s and don ts learn more  being a good user of the forums  learn more  community guidelines  from everyone at codecademy and this community  welcome   '

In [39]:
# to see last dictionary extracted from Comments
key_words_dict_scores

defaultdict(<function rake_nltk.rake.Rake._build_word_co_occurance_graph.<locals>.<lambda>()>,
            {'exercise': 1,
             'way': 1,
             'fix': 3,
             'themforgetting': 2,
             'knowing': 1,
             'referring': 1,
             'common': 2,
             'mistakes': 2,
             'keep': 1,
             'says': 3,
             'triangle': 27,
             'takes': 3,
             'formatted': 2,
             'mine': 2,
             'spelling': 2,
             'capitalization': 2,
             '__init__': 16,
             'method': 4,
             'typos': 4,
             'python': 7,
             'spell': 3,
             'things': 3,
             'exactly': 3,
             'one': 4,
             'argument': 4,
             'even': 4,
             'though': 4,
             'wrote': 3,
             'def': 21,
             'instead': 3,
             'made': 1,
             'code': 8,
             'selfvariable': 2,
             'values': 2,
   

In [40]:
# to see last item in Key_words
df['Key_words'][2]

['community',
 'guidelines',
 'moment',
 'take',
 'learn',
 'read',
 'introducing',
 'forums',
 'welcome',
 'categories',
 'help',
 'keep',
 'ts',
 'leaders',
 'organized',
 'everyone',
 'forum',
 'tidy',
 'good',
 'user',
 'bitesized',
 'posts',
 'codecademy']

In [41]:
# to combine 3 lists (3 columns) of key words into 1 sentence under Bag_of_words column
df['Bag_of_words'] = ''
columns = ['Category', 'Tags','Key_words']

for index, row in df.iterrows():
    words = ''
    for col in columns:
        words += ' '.join(row[col]) + ' '
    row['Bag_of_words'] = words
    
# strip white spaces infront and behind, replace multiple whitespaces (if any)
df['Bag_of_words'] = df['Bag_of_words'].str.strip().str.replace('   ', ' ').str.replace('  ', ' ')

df = df[['Topic Title','Bag_of_words']]
df

,Topic Title,Bag_of_words
0,[How to] Format code in posts,gettingstarted [ ] small amount visible either...
1,Guide: A list of how-to’s for giving and recei...,gettingstarted [ ] learn use format code forum...
2,Guide: Learn about our community forums,gettingstarted [ ] community guidelines moment...
3,[How-to] Create a topic that everyone will read,gettingstarted [ ] title icon get accurate muc...
4,[Learn More] How the forum is organized,gettingstarted [ ] peers subject mattertopic i...
...,...,...
5459,Why does it ask me not to use compute_bill?,faq [ ' p y t h o n f a q ' ] editor sucks nee...
5460,FAQ: Learn Python - Battleship - A Real Win,faq [ ' p y t h o n f a q ' ] new question con...
5461,How should I indent the code to create my_tria...,faq [ ' p y t h o n f a q ' ] create my_triang...
5462,FAQ: Learn Python - Battleship - Not Again!,faq [ ' p y t h o n f a q ' ] need new questio...


In [42]:
# to generate the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(df['Bag_of_words'])
count_matrix

<5464x68822 sparse matrix of type '<class 'numpy.int64'>'
	with 633481 stored elements in Compressed Sparse Row format>

In [43]:
# to generate the cosine similarity matrix (size 5464 x 5464)
# rows represent all topic-titles; columns represent all topic-titles
# cosine similarity: similarity = cos(angle) = range from 0 (different) to 1 (similar)
# all the numbers on the diagonal are 1 because every topic-title is identical to itself (cosine value is 1 means exactly identical)
# matrix is also symmetrical because the similarity between A and B is the same as the similarity between B and A.
# for other values eg 0.1578947, movie x and movie y has similarity value of 0.1578947

cosine_sim = cosine_similarity(count_matrix, count_matrix)
print(cosine_sim)

[[1.         0.12478355 0.07426107 ... 0.10890171 0.0562181  0.09490708]
 [0.12478355 1.         0.28005602 ... 0.06160411 0.0795044  0.03834825]
 [0.07426107 0.28005602 1.         ... 0.         0.08111071 0.04564355]
 ...
 [0.10890171 0.06160411 0.         ... 1.         0.10705187 0.18072435]
 [0.0562181  0.0795044  0.08111071 ... 0.10705187 1.         0.11846978]
 [0.09490708 0.03834825 0.04564355 ... 0.18072435 0.11846978 1.        ]]


In [44]:
# to create a Series for topic-titles which can be used as indices (each index is mapped to a topic-title)
indices = pd.Series(df['Topic Title'])
indices[:5]

0                        [How to] Format code in posts
1    Guide: A list of how-to’s for giving and recei...
2              Guide: Learn about our community forums
3      [How-to] Create a topic that everyone will read
4              [Learn More] How the forum is organized
Name: Topic Title, dtype: object

In [45]:
# this function takes in a topic-title as input and returns the top 10 recommended (similar) topic-titles

def recommend(title, cosine_sim = cosine_sim):
    recommended_posts = []
    idx = indices[indices == title].index[0]   # to get the index of the topic-title matching the input topic-title
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)   # similarity scores in descending order
    top_10_indices = list(score_series.iloc[1:11].index)   # to get the indices of top 10 most similar topic-title
    # [1:11] to exclude 0 (index 0 is the input topic-title itself)
    
    for i in top_10_indices:   # to append the titles of top 10 similar posts to the recommended_posts list
        recommended_posts.append(list(df['Topic Title'])[i])
        
    return recommended_posts

recommend('[How to] Format code in posts')

['Set up the albums#show controller action in the Bass Music project',
 'Bank teller project',
 'My Python function won’t run',
 'Orion Constellation Project',
 'New York City Blog Project: problem with <article>',
 'Censor Dispenser',
 'I’m having a problem with the Rock Paper Scissors JavaScript Project',
 'Best Fare Calculator',
 'Reverse a string code not working',
 '[How-to] Reply to a Post']

In [46]:
recommend('Guide: Learn about our community forums')

['[Learn more] Welcome to the Codecademy Community Forums!',
 'My gols',
 'Guide: A list of how-to’s for giving and receiving code-help',
 '[Learn more] Being a good user of the forums',
 'Data Structures',
 '[Learn more] About our community leaders',
 '[Learn More] How the forum is organized',
 'Anyone familiar with ruby?',
 'Forum',
 '[Learn more] Categories help keep the forum tidy']

In [47]:
recommend('[How-to] Create a topic that everyone will read')

['Is this the expected output on hurricane analysis mortality ratings:',
 'How to ask good questions (and get good answers)',
 'CoolTShirts',
 '4. Aliases I, step 2',
 'Str function',
 'New to java please explain code to me',
 'Multiple Tables with Reddit SQL Project Task 5',
 'If else statement Question! Super easy yet confusing',
 'Confused with Advance Python List challenge? Please help',
 'Page Visits Funnel Project Question']

In [48]:
recommend('[Learn More] How the forum is organized')

['[Learn more] Categories help keep the forum tidy',
 'Guide: Learn about our community forums',
 'My gols',
 'CoolTShirts',
 'Codeacademy down?',
 'Guide: A list of how-to’s for giving and receiving code-help',
 'App developers',
 '30 Days challenge',
 'Is this the expected output on hurricane analysis mortality ratings:',
 'Helpful books for coding and programming?']